Text infilling: context, T, attribute, A, and the value, V , in a sentence as “T. A is V .” where the attribute value V is masked as blank

Answer generation: generate V as the answer, considering T as the context and A as the question.

Answer generation seems to be plausible. I just train the a model on that. I will just use T5 because its not important for me to have the best performance. 

In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import(
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [6]:
from datasets import load_dataset
jnlpba = load_dataset('jnlpba', split=['train', "validation"])
print(jnlpba[0][0])

Found cached dataset jnlpba (/Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4)


  0%|          | 0/2 [00:00<?, ?it/s]

{'id': '1', 'tokens': ['IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.'], 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}


I need a df with question, context and answer like:

What are the entities?

IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production','by', '5-lipoxygenase', '.

attr1, attr2, attr3

The question I still have is how i need to tokenize everything - how i am going to do that?

In [8]:
#create dataframe with the 3 columns 

#create function which adds columns to df as i want 
df_train = pd.DataFrame(jnlpba[0])
print(df_train)
df_val = pd.DataFrame(jnlpba[1])
print(len(df_train))
# Create a mapping between the ner_tags values and the attribute values
mapping = {
    0: "O",
    1: "B-DNA",
    2: "I-DNA",
    3: "B-RNA",
    4: "I-RNA",
    5: "B-cell_line",
    6: "I-cell_line",
    7: "B-cell_type",
    8: "I-cell_type",
    9: "B-protein",
    10: "I-protein"
}

def replace_tags(row):
    row['ner_tags'] = ' '.join([mapping[tag] for tag in row['ner_tags']])
    return row

def join_text(row):
    row['tokens'] = ' '.join(row['tokens'])
    return row
    
df_train = df_train.apply(replace_tags, axis=1)
df_val = df_val.apply(replace_tags, axis=1)
print(len(df_train))
def combine_rows(group):
    new_row = {
        'id': group['id'].iloc[0],
        'tokens': ' '.join(group['tokens']),
        'ner_tags': ' '.join(group['ner_tags'])
    }
    return pd.Series(new_row)

df_train = df_train.groupby('id').apply(combine_rows)
df_val = df_val.groupby('id').apply(combine_rows)
print(len(df_train))
#add to data as context the column tokens and as answer the column ner_tags, answer is always the same "What are the attributes?"
data = {"context": df_train['tokens'], "question": "What are the attributes?", "answer": df_train['ner_tags']}



# Create the data frame using the pd.DataFrame constructor
df = pd.DataFrame(data)

#can i not just use the number representation? The reason why i dont want to do this is because perhaps there is some meaning hidden inside of the bios tag "gene" for example.

#lets get this fucking bios tag representation - before i would like to fix the gh bug-.

          id                                             tokens  \
0          1  [IL-2, gene, expression, and, NF-kappa, B, act...   
1          2  [Activation, of, the, CD28, surface, receptor,...   
2          3  [In, primary, T, lymphocytes, we, show, that, ...   
3          4  [Delineation, of, the, CD28, signaling, cascad...   
4          5  [Our, data, suggest, that, lipoxygenase, metab...   
...      ...                                                ...   
37089  37090  [This, stimulation, can, also, be, abolished, ...   
37090  37091  [This, gp160, treatment, adversely, affects, t...   
37091  37092  [Effects, similar, to, gp160, were, seen, with...   
37092  37093  [The, aberrant, activation, of, AP-1, by, gp16...   
37093  37094                                                 []   

                                                ner_tags  
0      [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]  
1      [0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0...  
2      [0, 7, 8, 8

KeyError: 'id'

### Toknization

In [ ]:
MODEL_NAME = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

/Users/maxhager/.virtualenvs/thesis/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
class NERDataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer: T5Tokenizer, answer_max_token_len, question_max_token_len, context_max_token_len):
        self.tokenizer = tokenizer
        self.data = data
        self.answer_max_token_len = data['context'].apply(len).max()
        self.question_max_token_len = data['answer'].apply(len).max()
        self.context_max_token_len = data['question'].apply(len).max()
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        data_row = self.data.iloc[index]
        encoding_context = tokenizer(
        data_row["context"].tolist(),
        max_length=512,
        padding=self.context_max_token_len,
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
        )
        
        encoding_answer = tokenizer(
        data_row["answer"].tolist(),
        padding=self.answer_max_token_len,
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"   
        )
        
        encoding_question = tokenizer(
        data_row["question"].tolist(),
        padding=self.question_max_token_len,
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
        )
        
        labels = encoding_answer['input_ids']
        labels[labels == 0] = -100
        
        return dict(
            question = data_row["question"],
            context = data_row["context"],
            answer = data_row["answer"],
            input_ids = encoding_context['input_ids'].flatten(),
            attention_mask = encoding_context['attention_mask'].flatten(),
            labels = labels.flatten()   
        )


In [ ]:
sample_dataset = NERDataset(df, tokenizer, max_length_answer, max_length_question, max_length_context)